# Day 56: Introduction to Recurrent Neural Networks (RNNs)

## Introduction

Welcome to Day 56 of the 100 Days of Machine Learning! Today, we embark on an exciting journey into the world of **Recurrent Neural Networks (RNNs)**, a class of neural networks specifically designed to handle sequential data. Unlike the feedforward neural networks and convolutional neural networks we've encountered before, RNNs possess a unique ability to maintain memory of previous inputs, making them ideal for tasks where context and order matter.

Sequential data is everywhere around us: the words in a sentence, stock prices over time, music notes in a melody, or the frames in a video. Traditional neural networks struggle with such data because they treat each input independently, losing the critical temporal or sequential relationships. RNNs solve this problem by introducing connections that loop back on themselves, allowing information to persist across time steps.

### Why RNNs Matter in Machine Learning

Recurrent Neural Networks have revolutionized how we approach sequential data problems. They've enabled breakthroughs in:
- **Natural Language Processing**: Machine translation, text generation, and sentiment analysis
- **Speech Recognition**: Converting spoken language to text
- **Time Series Forecasting**: Predicting stock prices, weather patterns, and energy consumption
- **Video Analysis**: Action recognition and video captioning
- **Music Generation**: Creating new musical compositions

The key innovation of RNNs is their ability to use their internal state (memory) to process sequences of inputs. This makes them fundamentally different from other neural network architectures and particularly powerful for temporal data.

### Learning Objectives

By the end of this lesson, you will be able to:
1. **Understand** the fundamental architecture and working principle of Recurrent Neural Networks
2. **Explain** the concept of hidden states and how information flows through time in RNNs
3. **Implement** basic RNN models using Python and modern deep learning frameworks
4. **Apply** RNNs to sequence prediction tasks and understand their strengths and limitations
5. **Recognize** common applications of RNNs in real-world scenarios

Let's dive in and discover how RNNs enable machines to understand and generate sequential patterns!

## Understanding Sequential Data

Before we dive into RNNs, it's essential to understand what makes sequential data special and why we need a different approach to handle it.

### What is Sequential Data?

Sequential data is any data where the order of observations matters. Unlike tabular data where each row is independent, sequential data has temporal or positional dependencies. For example:
- The sentence "I love machine learning" has a very different meaning than "learning machine love I"
- Yesterday's stock price influences today's price
- The previous notes in a melody affect which note comes next

### The Challenge for Traditional Neural Networks

Standard feedforward neural networks have two major limitations when dealing with sequential data:

1. **Fixed Input Size**: They require inputs of a fixed size, but sequences can vary in length
2. **No Memory**: They don't share information across different positions in the sequence

Consider trying to predict the next word in a sentence using a feedforward network. You'd need to decide in advance how many previous words to consider, and the network would learn completely separate parameters for each position. This is inefficient and doesn't capture the true sequential nature of language.

### Enter Recurrent Neural Networks

RNNs solve these problems by:
- Processing sequences one element at a time
- Maintaining a **hidden state** that acts as memory
- Sharing the same parameters across all time steps

This elegant design allows RNNs to handle sequences of any length while capturing temporal dependencies.

## RNN Architecture and Mathematical Foundation

### The Core Idea: Recurrence

The key innovation in RNNs is the introduction of a **recurrent connection**. At each time step $t$, an RNN takes two inputs:
1. The current input $x_t$
2. The previous hidden state $h_{t-1}$

It then produces two outputs:
1. The current hidden state $h_t$
2. The output $y_t$ (if required)

### Mathematical Formulation

The mathematical equations governing an RNN at time step $t$ are:

$$h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)$$

$$y_t = W_{hy} h_t + b_y$$

Where:
- $x_t \in \mathbb{R}^d$ is the input vector at time $t$
- $h_t \in \mathbb{R}^h$ is the hidden state vector at time $t$
- $y_t \in \mathbb{R}^o$ is the output vector at time $t$
- $W_{xh} \in \mathbb{R}^{h \times d}$ is the input-to-hidden weight matrix
- $W_{hh} \in \mathbb{R}^{h \times h}$ is the hidden-to-hidden weight matrix (this creates the recurrence!)
- $W_{hy} \in \mathbb{R}^{o \times h}$ is the hidden-to-output weight matrix
- $b_h \in \mathbb{R}^h$ and $b_y \in \mathbb{R}^o$ are bias vectors
- $\tanh$ is the hyperbolic tangent activation function

### Understanding the Hidden State

The **hidden state** $h_t$ is the memory of the RNN. It summarizes all the information from previous time steps that is relevant for the current and future predictions. Think of it as a compressed representation of the sequence history.

The hidden state is updated at each time step by combining:
- Information from the current input (via $W_{xh} x_t$)
- Information from the previous hidden state (via $W_{hh} h_{t-1}$)

This recursive update is what gives RNNs their power and their name!

### Unfolding Through Time

While we write the RNN equations for a single time step, during processing, the network is "unfolded" across the entire sequence. For a sequence of length $T$, we have:

$$h_1 = \tanh(W_{hh} h_0 + W_{xh} x_1 + b_h)$$
$$h_2 = \tanh(W_{hh} h_1 + W_{xh} x_2 + b_h)$$
$$\vdots$$
$$h_T = \tanh(W_{hh} h_{T-1} + W_{xh} x_T + b_h)$$

Notice that the **same weights** $W_{xh}$, $W_{hh}$, and $W_{hy}$ are used at every time step. This parameter sharing is crucial—it allows the network to generalize to sequences of any length.

### Backpropagation Through Time (BPTT)

Training RNNs requires a special version of backpropagation called **Backpropagation Through Time (BPTT)**. The idea is to unfold the RNN across all time steps and then apply standard backpropagation, with gradients flowing backward through time.

The loss function is typically computed as the sum of losses at each time step:

$$L = \sum_{t=1}^{T} L_t(y_t, \hat{y}_t)$$

Where $L_t$ is the loss at time $t$, $y_t$ is the true output, and $\hat{y}_t$ is the predicted output.

### Challenges: Vanishing and Exploding Gradients

RNNs face a significant challenge: as gradients are backpropagated through many time steps, they can either:
- **Vanish**: Become very small, making it hard to learn long-term dependencies
- **Explode**: Become very large, causing training instability

This happens because gradients are multiplied by the weight matrix $W_{hh}$ at each time step. If the largest eigenvalue of $W_{hh}$ is:
- Less than 1: gradients shrink exponentially (vanishing)
- Greater than 1: gradients grow exponentially (exploding)

We'll explore solutions to these problems (like LSTM and GRU) in upcoming lessons!

## Python Implementation

Now that we understand the theory, let's implement RNNs using Python. We'll use both NumPy for a simple implementation to understand the mechanics, and TensorFlow/Keras for practical applications.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

ModuleNotFoundError: No module named 'numpy'

### Simple RNN from Scratch

Let's build a minimal RNN from scratch using NumPy to understand exactly how it works. We'll create a simple character-level RNN that can learn to predict the next character in a sequence.

In [2]:
class SimpleRNN:
    """A simple RNN implementation from scratch"""
    
    def __init__(self, input_size, hidden_size, output_size):
        """
        Initialize the RNN with random weights.
        
        Parameters:
        - input_size: dimension of input vector
        - hidden_size: dimension of hidden state
        - output_size: dimension of output vector
        """
        self.hidden_size = hidden_size
        
        # Initialize weights with small random values
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01  # Input to hidden
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # Hidden to hidden
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  # Hidden to output
        
        # Initialize biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
    
    def forward(self, inputs, h_prev):
        """
        Forward pass through the RNN.
        
        Parameters:
        - inputs: sequence of input vectors
        - h_prev: initial hidden state
        
        Returns:
        - outputs: sequence of output vectors
        - hidden_states: sequence of hidden states
        """
        hidden_states = [h_prev]
        outputs = []
        
        # Process each time step
        for x in inputs:
            # Reshape input
            x = x.reshape(-1, 1)
            
            # Compute new hidden state: h_t = tanh(Whh * h_{t-1} + Wxh * x_t + bh)
            h = np.tanh(np.dot(self.Whh, h_prev) + np.dot(self.Wxh, x) + self.bh)
            
            # Compute output: y_t = Why * h_t + by
            y = np.dot(self.Why, h) + self.by
            
            # Store results
            hidden_states.append(h)
            outputs.append(y)
            
            # Update h_prev for next iteration
            h_prev = h
        
        return outputs, hidden_states

# Example: Create a simple RNN
input_size = 5
hidden_size = 10
output_size = 5
sequence_length = 3

rnn = SimpleRNN(input_size, hidden_size, output_size)

# Create a random input sequence
input_sequence = [np.random.randn(input_size) for _ in range(sequence_length)]
h0 = np.zeros((hidden_size, 1))  # Initial hidden state

# Forward pass
outputs, hidden_states = rnn.forward(input_sequence, h0)

print(f"Input sequence length: {len(input_sequence)}")
print(f"Number of outputs: {len(outputs)}")
print(f"Number of hidden states: {len(hidden_states)}")
print(f"\nOutput shape at each time step: {outputs[0].shape}")
print(f"Hidden state shape: {hidden_states[0].shape}")
print("\nSimple RNN forward pass completed successfully!")

NameError: name 'np' is not defined

### Visualizing Hidden State Evolution

Let's visualize how the hidden state evolves over time as the RNN processes a sequence.

In [3]:
# Create a longer sequence for better visualization
sequence_length = 20
input_sequence = [np.sin(i * 0.5) * np.ones(input_size) for i in range(sequence_length)]
h0 = np.zeros((hidden_size, 1))

# Run forward pass
outputs, hidden_states = rnn.forward(input_sequence, h0)

# Convert hidden states to array for visualization
hidden_array = np.array([h.flatten() for h in hidden_states[1:]])  # Skip initial state

# Create visualization
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Hidden state heatmap
im = axes[0].imshow(hidden_array.T, aspect='auto', cmap='RdBu', interpolation='nearest')
axes[0].set_xlabel('Time Step', fontsize=12)
axes[0].set_ylabel('Hidden Unit Index', fontsize=12)
axes[0].set_title('RNN Hidden State Evolution Over Time', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=axes[0], label='Activation Value')

# Plot 2: Selected hidden units over time
time_steps = np.arange(sequence_length)
for i in range(min(5, hidden_size)):  # Plot first 5 hidden units
    axes[1].plot(time_steps, hidden_array[:, i], marker='o', label=f'Hidden Unit {i}')
axes[1].set_xlabel('Time Step', fontsize=12)
axes[1].set_ylabel('Activation Value', fontsize=12)
axes[1].set_title('Individual Hidden Unit Activations Over Time', fontsize=14, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Visualization complete!")
print(f"We can see how the hidden state (RNN's memory) changes as it processes the sequence.")

NameError: name 'np' is not defined

## Practical Example: Sequence Prediction

Now let's use a modern deep learning framework (TensorFlow/Keras) to build an RNN for a practical task: learning to predict the next value in a simple sine wave pattern.

In [4]:
# Install TensorFlow if not already installed
try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("Installing TensorFlow...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow"])
    import tensorflow as tf
    print(f"TensorFlow installed! Version: {tf.__version__}")

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

print("TensorFlow and Keras ready!")

Installing TensorFlow...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 150.8 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.9/620.6 MB 6.7 MB/s eta 0:01:33

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/620.6 MB 6.4 MB/s eta 0:01:37

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/620.6 MB 5.7 MB/s eta 0:01:49

...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 6.5/7.0 MB 4.0 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 9.3 MB/s eta 0:00:00


2025-11-09 11:53:52.237012: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-09 11:53:52.237569: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 11:53:52.327399: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-11-09 11:53:56.241485: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-09 11:53:56.255505: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


TensorFlow installed! Version: 2.20.0
TensorFlow and Keras ready!


In [5]:
# Generate synthetic time series data (sine wave)
def generate_sine_wave(n_steps, n_sequences):
    """
    Generate sine wave sequences for training.
    
    Parameters:
    - n_steps: length of each sequence
    - n_sequences: number of sequences to generate
    """
    X, y = [], []
    
    for _ in range(n_sequences):
        # Random starting point
        start = np.random.rand() * 2 * np.pi
        # Generate sine wave
        sequence = np.sin(np.linspace(start, start + 4 * np.pi, n_steps + 1))
        # Input is all but last value, target is all but first value
        X.append(sequence[:-1].reshape(-1, 1))
        y.append(sequence[1:].reshape(-1, 1))
    
    return np.array(X), np.array(y)

# Generate training data
n_steps = 50  # sequence length
n_sequences_train = 1000
n_sequences_test = 200

X_train, y_train = generate_sine_wave(n_steps, n_sequences_train)
X_test, y_test = generate_sine_wave(n_steps, n_sequences_test)

print(f"Training data shape: X={X_train.shape}, y={y_train.shape}")
print(f"Test data shape: X={X_test.shape}, y={y_test.shape}")

# Visualize a few training sequences
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for i in range(4):
    axes[i].plot(X_train[i], label='Input', marker='o', markersize=3)
    axes[i].plot(y_train[i], label='Target (shifted)', marker='x', markersize=3)
    axes[i].set_title(f'Training Sequence {i+1}', fontweight='bold')
    axes[i].set_xlabel('Time Step')
    axes[i].set_ylabel('Value')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nData generation complete!")

NameError: name 'np' is not defined

In [6]:
# Build RNN model using Keras
model = Sequential([
    SimpleRNN(32, activation='tanh', return_sequences=True, input_shape=(n_steps, 1)),
    SimpleRNN(32, activation='tanh', return_sequences=True),
    Dense(1)
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Display model architecture
print("RNN Model Architecture:")
print("=" * 60)
model.summary()
print("=" * 60)

RNN Model Architecture:


2025-11-09 11:53:59.511272: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 50, 32)         │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 50, 32)         │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50, 1)          │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
print("Training RNN model...\n")

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

print("\nTraining complete!")

Training RNN model...



NameError: name 'X_train' is not defined

In [8]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Model Loss During Training', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE plot
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Mean Absolute Error', fontsize=12)
axes[1].set_title('Model MAE During Training', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Evaluate on test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Set Performance:")
print(f"  Loss (MSE): {test_loss:.6f}")
print(f"  MAE: {test_mae:.6f}")

NameError: name 'plt' is not defined

In [9]:
# Make predictions and visualize results
predictions = model.predict(X_test[:5], verbose=0)

# Plot predictions vs actual
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i in range(5):
    axes[i].plot(y_test[i], label='True Values', marker='o', markersize=4, linewidth=2)
    axes[i].plot(predictions[i], label='Predictions', marker='x', markersize=4, linewidth=2, linestyle='--')
    axes[i].set_title(f'Test Sequence {i+1}', fontweight='bold')
    axes[i].set_xlabel('Time Step')
    axes[i].set_ylabel('Value')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)
    
    # Calculate and display error for this sequence
    mse = np.mean((y_test[i] - predictions[i])**2)
    axes[i].text(0.02, 0.98, f'MSE: {mse:.4f}', 
                transform=axes[i].transAxes, 
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Remove the 6th subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

print("\nThe RNN has successfully learned to predict the sine wave pattern!")
print("Notice how the predictions closely follow the true values.")

NameError: name 'X_test' is not defined

## Hands-On Activity: Text Sequence Prediction

Now let's apply RNNs to a classic problem: character-level text generation. We'll train an RNN to learn patterns in text and predict the next character in a sequence.

This is a simplified version of what powers modern language models and autocomplete features!

In [10]:
# Sample text for training
text = """Machine learning is the study of computer algorithms that improve automatically through experience.
It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model
based on sample data, known as training data, in order to make predictions or decisions without being
explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications,
such as email filtering and computer vision, where it is difficult or infeasible to develop conventional
algorithms to perform the needed tasks."""

# Convert text to lowercase and get unique characters
text = text.lower()
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print(f"Text length: {len(text)} characters")
print(f"Unique characters: {len(chars)}")
print(f"Characters: {''.join(chars)}")

# Prepare sequences
seq_length = 40
step = 3  # Step size for creating sequences

sequences = []
next_chars = []

for i in range(0, len(text) - seq_length, step):
    sequences.append(text[i:i + seq_length])
    next_chars.append(text[i + seq_length])

print(f"\nNumber of sequences: {len(sequences)}")
print(f"\nExample sequence:")
print(f"Input: '{sequences[0]}'")
print(f"Target: '{next_chars[0]}'")

Text length: 556 characters
Unique characters: 27
Characters: 
 ,.abcdefghiklmnoprstuvwxy

Number of sequences: 172

Example sequence:
Input: 'machine learning is the study of compute'
Target: 'r'


In [11]:
# Vectorize the sequences
X_text = np.zeros((len(sequences), seq_length, len(chars)), dtype=np.float32)
y_text = np.zeros((len(sequences), len(chars)), dtype=np.float32)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X_text[i, t, char_to_idx[char]] = 1
    y_text[i, char_to_idx[next_chars[i]]] = 1

print(f"Input shape: {X_text.shape}")
print(f"Output shape: {y_text.shape}")
print(f"\nData prepared for character-level RNN!")

NameError: name 'np' is not defined

In [12]:
# Build character-level RNN model
text_model = Sequential([
    SimpleRNN(128, input_shape=(seq_length, len(chars)), activation='relu'),
    Dense(len(chars), activation='softmax')
])

text_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Character-level RNN Model:")
print("=" * 60)
text_model.summary()
print("=" * 60)

Character-level RNN Model:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        19,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 27)             │         3,483 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,451 (91.61 KB)

 Trainable params: 23,451 (91.61 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train the text model
print("Training character-level RNN...\n")

text_history = text_model.fit(
    X_text, y_text,
    epochs=100,
    batch_size=128,
    validation_split=0.1,
    verbose=0
)

# Plot training progress
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(text_history.history['loss'], label='Training', linewidth=2)
axes[0].plot(text_history.history['val_loss'], label='Validation', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Character-level RNN Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(text_history.history['accuracy'], label='Training', linewidth=2)
axes[1].plot(text_history.history['val_accuracy'], label='Validation', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Character-level RNN Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

final_acc = text_history.history['val_accuracy'][-1]
print(f"\nTraining complete! Final validation accuracy: {final_acc:.2%}")

Training character-level RNN...



NameError: name 'X_text' is not defined

In [14]:
# Generate text using the trained model
def generate_text(model, seed_text, length=100, temperature=1.0):
    """
    Generate text using the trained RNN.
    
    Parameters:
    - model: trained model
    - seed_text: initial text to start generation
    - length: number of characters to generate
    - temperature: controls randomness (higher = more random)
    """
    generated = seed_text
    
    for _ in range(length):
        # Prepare input
        x = np.zeros((1, seq_length, len(chars)))
        for t, char in enumerate(seed_text):
            if char in char_to_idx:
                x[0, t, char_to_idx[char]] = 1
        
        # Predict next character
        preds = model.predict(x, verbose=0)[0]
        
        # Apply temperature
        preds = np.log(preds + 1e-7) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))
        
        # Sample next character
        next_idx = np.random.choice(len(chars), p=preds)
        next_char = idx_to_char[next_idx]
        
        # Update generated text and seed
        generated += next_char
        seed_text = seed_text[1:] + next_char
    
    return generated

# Generate text with different seeds
seeds = [
    "machine learning is the study of comp",
    "algorithms that improve automatically "
]

print("Generated Text Samples:")
print("=" * 80)

for i, seed in enumerate(seeds, 1):
    generated = generate_text(text_model, seed, length=150, temperature=0.5)
    print(f"\nSample {i}:")
    print(f"Seed: '{seed}'")
    print(f"Generated: '{generated}'")
    print("-" * 80)

print("\nThe RNN has learned patterns in the text and can generate new sequences!")
print("While not perfect, it demonstrates the RNN's ability to model sequential dependencies.")

Generated Text Samples:


NameError: name 'np' is not defined

## Real-World Applications of RNNs

Recurrent Neural Networks have transformed numerous fields by enabling machines to understand and generate sequential data. Here are some key applications:

### 1. Natural Language Processing (NLP)
- **Machine Translation**: Translating text from one language to another (e.g., Google Translate)
- **Text Generation**: Creating human-like text for chatbots, story generation, and content creation
- **Sentiment Analysis**: Understanding emotions and opinions in customer reviews and social media
- **Named Entity Recognition**: Identifying people, places, and organizations in text

### 2. Speech and Audio Processing
- **Speech Recognition**: Converting spoken language to text (e.g., Siri, Alexa)
- **Speech Synthesis**: Generating natural-sounding speech from text
- **Music Generation**: Creating new musical compositions
- **Audio Classification**: Identifying sounds and speakers

### 3. Time Series Forecasting
- **Stock Price Prediction**: Forecasting financial market trends
- **Weather Forecasting**: Predicting future weather conditions
- **Energy Demand Prediction**: Forecasting electricity consumption
- **Sales Forecasting**: Predicting future product demand

### 4. Video Analysis
- **Action Recognition**: Identifying activities in videos
- **Video Captioning**: Generating textual descriptions of video content
- **Gesture Recognition**: Understanding human gestures for human-computer interaction

### 5. Healthcare
- **Medical Diagnosis**: Analyzing patient histories and sequential medical data
- **Drug Discovery**: Modeling molecular sequences for new drug development
- **Patient Monitoring**: Predicting health events from continuous monitoring data

### Limitations of Basic RNNs

While powerful, basic RNNs have some limitations:
1. **Vanishing/Exploding Gradients**: Difficulty learning long-term dependencies
2. **Computational Intensity**: Sequential processing limits parallelization
3. **Memory Limitations**: Simple hidden state may not capture complex long-term patterns

These limitations led to the development of more advanced architectures like **LSTM** (Long Short-Term Memory) and **GRU** (Gated Recurrent Units), which we'll explore in upcoming lessons!

## Key Takeaways

Congratulations on completing Day 56! Here are the essential points to remember:

1. **RNNs are designed for sequential data**: Unlike feedforward networks, RNNs maintain a hidden state that acts as memory, allowing them to process sequences of varying length and capture temporal dependencies.

2. **The hidden state is the key innovation**: At each time step, the RNN updates its hidden state by combining the current input with the previous hidden state, enabling information to flow through time.

3. **Parameter sharing across time steps**: RNNs use the same weights at every time step, which allows them to generalize to sequences of any length and reduces the number of parameters.

4. **Training uses Backpropagation Through Time (BPTT)**: Gradients flow backward through the unfolded network, but this can lead to vanishing or exploding gradient problems for long sequences.

5. **Wide range of applications**: RNNs excel at tasks involving sequential data, including natural language processing, speech recognition, time series forecasting, and video analysis.

6. **Limitations exist**: Basic RNNs struggle with long-term dependencies due to the vanishing gradient problem, which motivates more advanced architectures like LSTM and GRU.

### What You Should Now Be Able To Do:
- Explain how RNNs differ from feedforward neural networks
- Understand the mathematics behind RNN forward propagation
- Implement basic RNNs using both NumPy and TensorFlow/Keras
- Apply RNNs to sequence prediction tasks
- Recognize appropriate use cases for RNNs in real-world applications

In the next lessons, we'll explore LSTM and GRU networks, which address the limitations of basic RNNs and enable learning from much longer sequences!

## Further Resources

To deepen your understanding of Recurrent Neural Networks, explore these resources:

### Academic Papers
1. **"Finding Structure in Time" by Elman (1990)**: The seminal paper introducing recurrent networks
   - [Link to paper](https://crl.ucsd.edu/~elman/Papers/fsit.pdf)

2. **"Learning representations by back-propagating errors" by Rumelhart, Hinton, and Williams (1986)**: Foundation of backpropagation
   - Essential for understanding BPTT

### Online Tutorials and Courses
3. **Andrej Karpathy's "The Unreasonable Effectiveness of Recurrent Neural Networks"**: Excellent blog post with visualizations
   - [Link to blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

4. **Colah's Blog: "Understanding LSTM Networks"**: Clear visual explanations
   - [Link to blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

5. **TensorFlow RNN Tutorial**: Official TensorFlow documentation and tutorials
   - [Link to docs](https://www.tensorflow.org/guide/keras/rnn)

### Books
6. **"Deep Learning" by Goodfellow, Bengio, and Courville**: Chapter 10 covers sequence modeling
   - Available free online: [Link](https://www.deeplearningbook.org/)

7. **"Neural Networks and Deep Learning" by Michael Nielsen**: Interactive online book
   - [Link to book](http://neuralnetworksanddeeplearning.com/)

### Practice Datasets
- **Penn Treebank**: Standard dataset for language modeling
- **IMDB Reviews**: For sentiment analysis with RNNs
- **Time Series datasets from UCI**: Various time series prediction tasks

Keep exploring and experimenting with RNNs. The best way to learn is by building your own models and applying them to problems you're interested in!